In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
import pickle

# Função para normalizar o texto
def normalizar_texto(texto):
    return texto.strip().lower()

# Carregar o dataset
dataset = pd.read_csv('problemas_combinados_2.csv', sep=',')

# Verificar as primeiras linhas do dataset
print("Primeiras linhas do dataset original:")
print(dataset.head())

# Unificar as colunas de quilometragem em uma única coluna
dataset_melted = pd.melt(
    dataset,
    id_vars=['Problema', 'Possíveis causas'],
    value_vars=['20.000km', '50.000km', '100.000km+'],
    var_name='Quilometragem',
    value_name='Probabilidade'
)

# Manter apenas as linhas com 'Probabilidade' > 0
dataset_melted = dataset_melted[dataset_melted['Probabilidade'] > 0]

# Normalizar os textos
dataset_melted['Problema'] = dataset_melted['Problema'].apply(normalizar_texto)
dataset_melted['Quilometragem'] = dataset_melted['Quilometragem'].apply(normalizar_texto)
dataset_melted['Possíveis causas'] = dataset_melted['Possíveis causas'].apply(
    lambda x: [normalizar_texto(causa) for causa in x.split(', ')]
)

# Combinar 'Problema' e 'Quilometragem' em uma única coluna de texto
dataset_melted['Entrada'] = dataset_melted['Problema'] + ' ' + dataset_melted['Quilometragem']

# Vetorizar a entrada usando TF-IDF com n-grams e ajustes
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=1,
    max_df=0.8,
    strip_accents='unicode',
    lowercase=True
)
X = vectorizer.fit_transform(dataset_melted['Entrada'])

# Binarizar 'Possíveis causas' para classificação multilabel
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(dataset_melted['Possíveis causas'])

# Verificar a distribuição das classes
print("\nDistribuição das classes após binarização:")
print(pd.DataFrame(y, columns=mlb.classes_).sum())

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Inicializar o modelo com OneVsRestClassifier e LogisticRegression com class_weight='balanced'
model = OneVsRestClassifier(
    LogisticRegression(max_iter=2000, solver='liblinear', class_weight='balanced')
)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

# Salvar o modelo, o vetorizador e o binarizador em um arquivo pickle
with open('modelo_causas_aprimorado.pkl', 'wb') as f:
    pickle.dump({
        'model': model,
        'vectorizer': vectorizer,
        'mlb': mlb
    }, f)

print("Modelo aprimorado salvo como 'modelo_causas_aprimorado.pkl'")

Primeiras linhas do dataset original:
                       Problema  \
0         Ar condicionado fraco   
1  Ar-condicionado não funciona   
2     Barulho estranho no motor   
3              Barulho no motor   
4    Carro puxando para um lado   

                                    Possíveis causas  20.000km  50.000km  \
0  Filtro sujo, Compressor com mal funcionamento,...       0.3       0.5   
1                              Filtro de ar entupido       0.2       0.4   
2  Correia dentada desgastada, Problemas no alter...       0.2       0.5   
3  Pressão incorreta, Correia dentada desgastada,...       0.2       0.4   
4        Desalinhamento, Pressão dos pneus desiguais       0.3       0.5   

   100.000km+  
0         0.7  
1         0.6  
2         0.8  
3         0.7  
4         0.6  

Distribuição das classes após binarização:
acúmulo de sujeira                           3
alinhamento da porta incorreto               3
alinhamento incorreto                        9
alternador co

d:\Projects\SecureCar\ia-sc\api-ml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Projects\SecureCar\ia-sc\api-ml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Projects\SecureCar\ia-sc\api-ml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pickle

# Carregar o modelo e os transformadores aprimorados
with open('modelo_causas_aprimorado.pkl', 'rb') as f:
    data = pickle.load(f)
    model = data['model']
    vectorizer = data['vectorizer']
    mlb = data['mlb']

# Função para normalizar o texto
def normalizar_texto(texto):
    return texto.strip().lower()

# Função para fazer predições
def prever_causas(problema_input, quilometragem_input):
    # Normalizar entradas
    problema_input_norm = normalizar_texto(problema_input)
    quilometragem_input_norm = normalizar_texto(quilometragem_input)
    
    # Criar a entrada combinada
    entrada = problema_input_norm + ' ' + quilometragem_input_norm

    # Transformar a entrada
    X_novo = vectorizer.transform([entrada])

    # Fazer a previsão
    y_pred = model.predict(X_novo)

    # Decodificar as causas previstas
    causas_previstas = mlb.inverse_transform(y_pred)

    # Verificar se há causas previstas
    if causas_previstas and causas_previstas[0]:
        return f"Possíveis causas para {str(problema_input).lower} com quilometragem {quilometragem_input} são: {'\n'.join(causas_previstas[0])} "
        # print(f"Possíveis causas para '{problema_input}' com quilometragem '{quilometragem_input}':")
        # for causa in causas_previstas[0]:
        #     print(f"- {causa}")
    else:
        return f"Nenhuma causa prevista para '{problema_input}' com quilometragem '{quilometragem_input}'."
        # print(f"Nenhuma causa prevista para '{problema_input}' com quilometragem '{quilometragem_input}'.")




Possíveis causas para 'Barulho no motor' com quilometragem '50.000km':
- correia dentada desgastada
- pressão incorreta
- óleo baixo


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/prever', methods=['POST'])
def prever():
    problema = request.json()['problema']
    quilometragem = str(request.json()['quilometragem']) + 'km'
    resultado = prever_causas(problema, quilometragem)
    return jsonify(resultado)



# Exemplo de uso com valores presentes no dataset
# problema = 'Barulho no motor'
# quilometragem = '50.000km'
